## Step 1a - Import modules/libraries

In [2]:
import os                             # a module for interacting with the operating system
import glob                           # used to retrieve files/pathnames matching a specified pattern
master_dr = os.path.split(os.getcwd())[0]
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
os.chdir(os.path.join(master_dr, "Modules"))
from GIS_functions import GIS_function as gis

# prepairations

In [6]:
roi_shapefile= os.path.join(master_dr, r"Data/1Boundary/Shapefile/wheat.geojson")
gdf=gpd.read_file(roi_shapefile) #read shapefile
resample_dr = os.path.join(master_dr, r"Data/tif/resample")

# tifs meta
source_file   = os.path.join(master_dr, r"Data/tif/download/WAPOR.v2_dekadal_L2_AETI_D/L2_AETI_1431.tif")
src = rasterio.open(source_file, dtype=np.float32)
tif_crop_array, tif_crop_transform = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata=0)
meta = src.meta
meta.update(
  dtype = np.float32,
  nodata= np.nan,
  width= tif_crop_array.shape[-1], 
  height= tif_crop_array.shape[-2],
  transform= tif_crop_transform,
  count= 1,
)

# seasonal data values

In [7]:
seasonal_dr = os.path.join(master_dr, r"Data/tif/seasonal")
os.makedirs(seasonal_dr, exist_ok= True)

for data_dr in sorted(os.listdir(resample_dr)):
  if 'LCC' in data_dr:  # exclude landcover
    continue
  data_name = data_dr.split('_')[-2]
  print(f"data = {data_name}") 
  target_fhs = sorted(glob.glob(os.path.join(resample_dr, data_dr)+'/*.tif'))
  output_file = os.path.join(seasonal_dr,f"seasonal_{data_name}.tif")
  season_array = 0
  for tif in target_fhs:
    src = rasterio.open(tif, dtype=np.float32)
    tif_crop_array, tif_crop_transform = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata=np.nan)
    season_array += tif_crop_array[0]
  with rasterio.open(output_file, 'w', **meta) as dst:
    dst.write(season_array, 1)

data = PCP
data = RET
data = AETI
data = NPP
data = T


# monthly data values

In [8]:
monthly_dr = os.path.join(master_dr, r"Data/tif/monthly")

for data_dr in sorted(os.listdir(resample_dr)):
  if 'LCC' in data_dr:  # exclude landcover
    continue
  data_name = data_dr.split('_')[-2]
  # print(f"data = {data_name}")
  target_fhs = sorted(glob.glob(os.path.join(resample_dr, data_dr)+'/*.tif'))
  current_month = round((int(os.path.basename(target_fhs[0]).split('_')[-1][2:4])+1)/3)
  current_year = f"20{os.path.basename(target_fhs[0]).split('_')[-1][:2]}"
  data_dr = os.path.join(monthly_dr, data_name)
  os.makedirs(data_dr, exist_ok= True)
  month_array = 0
  for i,tif in enumerate(target_fhs):
    src = rasterio.open(tif, dtype=np.float32)
    tif_crop_array, tif_crop_transform = rasterio.mask.mask(src, gdf.geometry, crop=True, filled=True, nodata=np.nan)
    # print(int(os.path.basename(tif).split('_')[-1][2:4]))
    month = round((int(os.path.basename(tif).split('_')[-1][2:4])+1)/3)
    # print("trail")
    if month == current_month:
      # print(f"m 1 {month}")
      month_array += tif_crop_array[0]
    if month != current_month or tif == target_fhs[-1]:
      # print(f"m 2 {month}")
      output_file = os.path.join(data_dr,f"monthly_{data_name}_{current_year}_{str(current_month).zfill(2)}.tif")
      with rasterio.open(output_file, 'w', **meta) as dst:
        dst.write(month_array, 1)

      current_year = f"20{os.path.basename(tif).split('_')[-1][:2]}"
      current_month = month
      month_array = 0
      month_array += tif_crop_array[0]